In [1]:
from bs4 import BeautifulSoup
import sys
import urllib
import urllib.request
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
from socket import timeout
import logging
import json
from pprint import pprint

In [2]:
try:
    response_BK = urllib.request.urlopen('http://streeteasy.com/for-rent/brooklyn', timeout=10).read().decode('utf-8')
    response_QN = urllib.request.urlopen('http://streeteasy.com/for-rent/queens', timeout=10).read().decode('utf-8')
    response_BX = urllib.request.urlopen('http://streeteasy.com/for-rent/bronx', timeout=10).read().decode('utf-8')
except (HTTPError, URLError) as error:
    logging.error('Data of %s not retrieved because %s\nURL: %s', name, error, url)
except timeout:
    logging.error('socket timed out - URL %s', url)
else:
    logging.info('Access successful.')

In [3]:
#Check if the website is working for brooklyn
#print(response_BK)

In [4]:
#Check if the website is working for queens
#print(response_QN)

In [5]:
#Check if the website is working for the Bronx
#print(response_BX)

In [6]:
def rent_Brooklyn(url):
    return 'http://streeteasy.com/for-rent/brooklyn?page=' + url

In [7]:
def rent_Queens(url):
    return 'http://streeteasy.com/for-rent/queens?page=' + url

In [8]:
def rent_Bronx(page):
    return 'http://streeteasy.com/for-rent/bronx?page=' + url

In [9]:
#for Brooklyn
price=[]
where=[]
bed=[]
bath=[]
size=[]
street=[]

In [10]:
#There is another way to parse the HTML of a website
#I am printing soup to help me find out the class ids
#go to https://streeteasy.com/home/feed_format for attributs
#Brooklyn Rent
for p in range(1,100): #loop through all pages
    url=rent_Brooklyn(str(p))
    urls = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(urls,'html.parser')
    #How to Pretty print html into a file
    #Which helps me to get details of classes
    #print(soup.prettify())
    #print(soup)
    y = lambda tag: tag.has_attr('data-id')
    BK_listing = soup.find_all(y)

    #print('aaaaa')
    for i in range(len(BK_listing)):

        #Find details title by street
        street.append(BK_listing[i].find_all('div',{'class':'details-title'})[0].a.string)
        #print('aaaaakkkkkkk')
        # Loop through brooklyn rent prices
        if BK_listing[i].find_all('span',{'class':'price'})!=[]:
            price.append(BK_listing[i].find_all('span',{'class':'price'})[0].string)
        else:
            price.append('')
        #print('aaaaappppppp')
        #Location of brooklyn neighborhoods
        #length=len(lst[i].find_all('div',{'class':'details_info'}))
        l = BK_listing[i].find_all('div',{'class':'details_info'})
        if(BK_listing[i].find_all('div',{'class':'details_info'})[0].find_all('a',href=True)!=[]):
            where.append(BK_listing[i].find_all('div',{'class':'details_info'})[0].find_all('a',href=True)[0].string)
        else:    
            if(len(l)!=1):
                if(BK_listing[i].find_all('div',{'class':'details_info'})[1].find_all('a',href=True)==[]):
                    where.append('')
                else:
                    where.append(BK_listing[i].find_all('div',{'class':'details_info'})[1].find_all('a',href=True)[0].string)
            else:
                where.append('')
        #print('aaaaaooooooo')
        #The size of the apartment
        if(BK_listing[i].find_all('span',{'class':'last_detail_cell'})!=[]):
            size.append(BK_listing[i].find_all('span',{'class':'last_detail_cell'})[0].string)
        else:
            size.append('')

        #loop through bedroom bath
        if(BK_listing[i].find_all('span',{'class':'detail_cell'})!=[]):
            bath.append(BK_listing[i].find_all('span',{'class':'detail_cell'})[0].string)
        else:
            bath.append('')

        #loop through bedroom bed
        if(BK_listing[i].find_all('span',{'class':'first_detail_cell'})!=[]):
            bed.append(BK_listing[i].find_all('span',{'class':'first_detail_cell'})[0].string)
        else:
            bed.append('')               

In [11]:
    #We will do a for loop for the other two boroughs
    #By using the pprint module you can examine the contents of an element.

    #dataframe for brooklyn
    data={'street':street,'price':price,'where':where,'bed':bed, 'bath':bath, 'size':size,'furnished':0}
    data=pd.DataFrame(data)

    #is the apartment furnished?
    cond=data['bed']=='Furnished'
    data.loc[cond,'furnished']=1
    data.loc[cond,'bed']=''

    #move from size to bath
    cond=[]
    for i in data['size']:
        if(i==''):
            cond.append(False)
        else:
            cond.append(i.split(" ")[1] in ('bath','baths'))
    data.loc[cond,'bath']=data.loc[cond,'size'] 
    data.loc[cond,'size']=''

    #move from bed to bath
    cond=[]
    for i in data['bed']:
        if(i=='' or i=='Furnished' or i=='studio'):
            cond.append(False)
        else:
            cond.append(i.split(" ")[1] in ('bath','baths'))
    data.loc[cond,'bath']=data.loc[cond,'bed'] 
    data.loc[cond,'bed']=''

    #move from bath to bed
    cond=[]
    for i in data['bath']:
        if(i==''):
            cond.append(False)
        else:
            if(len(i.split(" "))==1):
                cond.append(True)
            else:
                if(i.split(" ")[1] in ('bath','baths')):
                    cond.append(False)
                else:
                    cond.append(True)
    data.loc[cond,'bed']=data.loc[cond,'bath'] 
    data.loc[cond,'bath']=''

    #move from bed to size
    cond=[]
    for i in data['bed']:
        if(i=='' or i=='studio'):
            cond.append(False)
        else:
            if(i.split(" ")[1] in ('bed','beds')):
                cond.append(False)
            else:
                cond.append(True)
    data.loc[cond,'size']=data.loc[cond,'bed'] 
    data.loc[cond,'bed']=''


    #replace blank with nan
    data=data.applymap(lambda x: np.nan if x=='' else x)

    #data
    data.to_csv('data/Brooklyn_Rent.csv')
 
    print('Brooklyn Rent is done')
    data.head()

Brooklyn Rent is done


,bath,bed,furnished,price,size,street,where
0,1 bath,1.5 beds,0,"$2,500",NaN,93 Greenpoint Avenue #2R,Greenpoint
1,1 bath,2 beds,0,"$2,370","1,300 ft²",230 Hancock Street #1,Bedford-Stuyvesant
2,2 baths,2 beds,0,"$5,295","1,052 ft²",429 Kent Avenue #305,Williamsburg
3,1 bath,1 bed,0,"$3,600",766 ft²,429 Kent Avenue #306,Williamsburg
4,1 bath,2 beds,0,"$2,000",900 ft²,1028 Bergen Street #4,NaN


In [12]:
#for Queens
price=[]
where=[]
bed=[]
bath=[]
size=[]
street=[]
#There is another way to parse the HTML of a website
#I am printing soup to help me find out the class ids
#go to https://streeteasy.com/home/feed_format for attributs
for p in range(1,100): #loop through all pages
    url=rent_Queens(str(p))
    urls = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(urls,'html.parser')
    #How to Pretty print html into a file
    #Which helps me to get details of classes
    #print(soup.prettify())
    #print(soup)
    y = lambda tag: tag.has_attr('data-id')
    QN_listing = soup.find_all(y)

    #print('aaaaa')
    for i in range(len(QN_listing)):

        #Find details title by street
        street.append(QN_listing[i].find_all('div',{'class':'details-title'})[0].a.string)
        #print('aaaaakkkkkkk')
        # Loop through brooklyn rent prices
        if QN_listing[i].find_all('span',{'class':'price'})!=[]:
            price.append(QN_listing[i].find_all('span',{'class':'price'})[0].string)
        else:
            price.append('')
        #print('aaaaappppppp')
        #Location of brooklyn neighborhoods
        #length=len(lst[i].find_all('div',{'class':'details_info'}))
        l = QN_listing[i].find_all('div',{'class':'details_info'})
        if(QN_listing[i].find_all('div',{'class':'details_info'})[0].find_all('a',href=True)!=[]):
            where.append(QN_listing[i].find_all('div',{'class':'details_info'})[0].find_all('a',href=True)[0].string)
        else:    
            if(len(l)!=1):
                if(QN_listing[i].find_all('div',{'class':'details_info'})[1].find_all('a',href=True)==[]):
                    where.append('')
                else:
                    where.append(QN_listing[i].find_all('div',{'class':'details_info'})[1].find_all('a',href=True)[0].string)
            else:
                where.append('')
        #print('aaaaaooooooo')
        #The size of the apartment
        if(QN_listing[i].find_all('span',{'class':'last_detail_cell'})!=[]):
            size.append(QN_listing[i].find_all('span',{'class':'last_detail_cell'})[0].string)
        else:
            size.append('')

        #loop through bedroom bath
        if(QN_listing[i].find_all('span',{'class':'detail_cell'})!=[]):
            bath.append(QN_listing[i].find_all('span',{'class':'detail_cell'})[0].string)
        else:
            bath.append('')

        #loop through bedroom bed
        if(QN_listing[i].find_all('span',{'class':'first_detail_cell'})!=[]):
            bed.append(QN_listing[i].find_all('span',{'class':'first_detail_cell'})[0].string)
        else:
            bed.append('')
                       

In [13]:
    #dataframe for queens
    data2={'street':street,'price':price,'where':where,'bed':bed, 'bath':bath, 'size':size,'furnished':0}
    data2=pd.DataFrame(data2)

    #is the apartment furnished?
    cond=data2['bed']=='Furnished'
    data2.loc[cond,'furnished']=1
    data2.loc[cond,'bed']=''

    #move from size to bath
    cond=[]
    for i in data2['size']:
        if(i==''):
            cond.append(False)
        else:
            cond.append(i.split(" ")[1] in ('bath','baths'))
    data2.loc[cond,'bath']=data2.loc[cond,'size'] 
    data2.loc[cond,'size']=''

    #move from bed to bath
    cond=[]
    for i in data2['bed']:
        if(i=='' or i=='Furnished' or i=='studio'):
            cond.append(False)
        else:
            cond.append(i.split(" ")[1] in ('bath','baths'))
    data2.loc[cond,'bath']=data2.loc[cond,'bed'] 
    data2.loc[cond,'bed']=''

    #move from bath to bed
    cond=[]
    for i in data2['bath']:
        if(i==''):
            cond.append(False)
        else:
            if(len(i.split(" "))==1):
                cond.append(True)
            else:
                if(i.split(" ")[1] in ('bath','baths')):
                    cond.append(False)
                else:
                    cond.append(True)
    data2.loc[cond,'bed']=data2.loc[cond,'bath'] 
    data2.loc[cond,'bath']=''

    #move from bed to size
    cond=[]
    for i in data2['bed']:
        if(i=='' or i=='studio'):
            cond.append(False)
        else:
            if(i.split(" ")[1] in ('bed','beds')):
                cond.append(False)
            else:
                cond.append(True)
    data2.loc[cond,'size']=data2.loc[cond,'bed'] 
    data2.loc[cond,'bed']=''


    #replace blank with nan
    data2=data2.applymap(lambda x: np.nan if x=='' else x)

    #data
    data2.to_csv('data/Queens_rent.csv')